In [241]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [242]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



219

In [243]:
# determine winner based on match_id
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [244]:
#define heroes df
heroes_df = pd.read_json('working/heroes.json')
heroes_df = heroes_df[['id','localized_name','url']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name,url
0,1,Anti-Mage,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/antimage.png
1,2,Axe,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/axe.png
2,3,Bane,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bane.png
3,4,Bloodseeker,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bloodseeker.png
4,5,Crystal Maiden,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/crystal_maiden.png


In [245]:
# define players df
ti_teams_df= pd.read_json('working/teams.json')
ti_teams = list(ti_teams_df['team_id'])
players_df = pd.read_json('working/proPlayers.json')
players_df = players_df[['account_id','personaname','name','team_id','team_name','team_tag']]
players_df = players_df[players_df['team_id'].isin(ti_teams)]
players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra


In [246]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)

In [247]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df.head()

,id,hero_name,url,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/antimage.png,2.0,1.0,50.00,1.0
1,2,Axe,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/axe.png,5.0,2.0,40.00,2.0
2,3,Bane,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bane.png,8.0,4.0,50.00,3.0
3,4,Bloodseeker,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bloodseeker.png,12.0,6.0,50.00,58.0
4,5,Crystal Maiden,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/crystal_maiden.png,62.0,38.0,61.29,15.0


In [248]:
#create hero stats
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
players_teams = players_df.set_index('account_id').to_dict()['team_id']
hero_stats_df['team_id'] = hero_stats_df['account_id'].map(players_teams) 

hero_stats_df.head()

,match_id,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min,team_id
0,6806663153,86799300,102,0,4,7,51,361,222,7390454
1,6806663153,221666230,52,6,4,5,300,578,526,7390454
2,6806663153,87063175,137,6,4,6,178,462,378,7390454
3,6806663153,108452107,10,2,0,1,331,647,535,7390454
4,6806663153,86726887,128,2,6,10,42,301,187,7390454


In [249]:
hero_avg_stats = hero_stats_df.groupby('hero_id').mean()
hero_max_stats = hero_stats_df.groupby('hero_id').max()
hero_min_stats = hero_stats_df.groupby('hero_id').min()

heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_stats['kills'])

heroes_df['max_kills'] = heroes_df['id'].map(hero_max_stats['kills'])

heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_stats['deaths'])

heroes_df['min_deaths'] = heroes_df['id'].map(hero_min_stats['deaths'])

heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_stats['assists'])

heroes_df['max_assists'] = heroes_df['id'].map(hero_max_stats['assists'])

heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_stats['last_hits'])

heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_stats['last_hits'])

heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_stats['xp_per_min'])

heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_stats['gold_per_min'])

heroes_df.head()

,id,hero_name,url,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/antimage.png,2.0,1.0,50.00,1.0,5.500000,7.0,5.000000,4.0,9.000000,13.0,496.500000,522.0,853.500000,694.500000
1,2,Axe,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/axe.png,5.0,2.0,40.00,2.0,7.600000,20.0,4.200000,0.0,5.600000,10.0,190.400000,236.0,575.200000,458.600000
2,3,Bane,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bane.png,8.0,4.0,50.00,3.0,1.000000,4.0,5.125000,2.0,12.625000,20.0,45.625000,146.0,395.375000,264.125000
3,4,Bloodseeker,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bloodseeker.png,12.0,6.0,50.00,58.0,12.166667,25.0,3.000000,0.0,9.500000,16.0,379.916667,573.0,735.916667,636.750000
4,5,Crystal Maiden,https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/crystal_maiden.png,62.0,38.0,61.29,15.0,2.306452,5.0,6.290323,1.0,14.548387,28.0,62.612903,146.0,409.919355,288.532258


In [250]:
heroes_5_df = heroes_df[heroes_df['pick_count']>=5]

In [251]:
player_avg_stats = hero_stats_df.groupby('account_id').mean()
player_max_stats = hero_stats_df.groupby('account_id').max()
player_min_stats = hero_stats_df.groupby('account_id').min()

players_df['avg_kills'] = players_df['account_id'].map(player_avg_stats['kills'])

players_df['max_kills'] = players_df['account_id'].map(player_max_stats['kills'])

players_df['avg_deaths'] = players_df['account_id'].map(player_avg_stats['deaths'])

players_df['min_deaths'] = players_df['account_id'].map(player_min_stats['deaths'])

players_df['avg_assists'] = players_df['account_id'].map(player_avg_stats['assists'])

players_df['max_assists'] = players_df['account_id'].map(player_max_stats['assists'])

players_df['avg_last_hits'] = players_df['account_id'].map(player_avg_stats['last_hits'])

players_df['max_last_hits'] = players_df['account_id'].map(player_max_stats['last_hits'])

players_df['avg_xp_per_min'] = players_df['account_id'].map(player_avg_stats['xp_per_min'])

players_df['avg_gold_per_min'] = players_df['account_id'].map(player_avg_stats['gold_per_min'])

players_df['max_gold_per_min'] = players_df['account_id'].map(player_max_stats['gold_per_min'])

player_unique_heroes = hero_stats_df[['account_id','hero_id']].groupby('account_id').nunique()
players_df['unique_heroes'] = players_df['account_id'].map(player_unique_heroes['hero_id'])

players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min,max_gold_per_min,unique_heroes
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra,3.227273,6.0,5.363636,1.0,15.590909,29.0,78.772727,151.0,480.090909,327.545455,412.0,9.0
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG,2.863636,7.0,5.545455,1.0,14.318182,38.0,81.409091,156.0,436.772727,306.454545,395.0,9.0
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG,3.772727,12.0,5.818182,0.0,13.500000,23.0,51.272727,126.0,438.000000,288.772727,376.0,10.0
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX,3.388889,15.0,6.888889,2.0,10.611111,23.0,77.166667,202.0,424.944444,283.888889,423.0,11.0
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
ti_teams_df = ti_teams_df[['team_id','name','tag','url']]
teams_stats_df = hero_stats_df.groupby(['match_id','team_id']).sum()
team_avg_stats_df = teams_stats_df.groupby('team_id').mean()
team_max_stats_df = teams_stats_df.groupby('team_id').max()
team_min_stats_df = teams_stats_df.groupby('team_id').min()

ti_teams_df['avg_kills'] = ti_teams_df['team_id'].map(team_avg_stats_df['kills'])
ti_teams_df['avg_deaths'] = ti_teams_df['team_id'].map(team_avg_stats_df['deaths'])
ti_teams_df['avg_assists'] = ti_teams_df['team_id'].map(team_avg_stats_df['assists'])
ti_teams_df['max_kills'] = ti_teams_df['team_id'].map(team_max_stats_df['kills'])
ti_teams_df['min_deaths'] = ti_teams_df['team_id'].map(team_min_stats_df['deaths'])
ti_teams_df['max_assists'] = ti_teams_df['team_id'].map(team_max_stats_df['assists'])

team_unique_heroes = hero_stats_df[['team_id','hero_id']].groupby('team_id').nunique()
ti_teams_df['unique_heroes'] = ti_teams_df['team_id'].map(team_unique_heroes['hero_id'])
ti_teams_df.head()

,team_id,name,tag,url,avg_kills,avg_deaths,avg_assists,max_kills,min_deaths,max_assists,unique_heroes
0,8597976,Talon,TLN,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/8597976.png,22.000000,29.100000,51.650000,46,10,98,51
1,7732977,BOOM Esports,BOOM,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/7732977.png,21.782609,20.913043,53.434783,53,4,132,48
2,7390454,Soniqs,SQ,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/7390454.png,21.263158,25.210526,47.631579,44,3,112,51
3,2163,Team Liquid,Liquid,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/2163.png,29.250000,24.678571,67.928571,48,6,113,49
4,8260983,TSM FTX,TSM FTX,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/8260983.png,20.055556,27.277778,44.222222,44,3,99,48


In [297]:
def get_matchwinner(row):
    if row['radiant_win'] == True:
        winner = 'radiant_team_id'
    else:
        winner = 'dire_team_id'
    return row[winner]

def convert(seconds):
    # seconds = row[column]
    minutes = seconds // 60
    seconds = seconds % 60
    
     
    return "%02d:%02d" % (minutes, seconds)

ti_matches_df = matches_df[['match_id','duration','radiant_win','radiant_score','radiant_team_id','dire_score','dire_team_id']]
ti_matches_df['total_score'] = ti_matches_df['radiant_score']+ti_matches_df['dire_score']

ti_matches_df['winner'] = ti_matches_df.apply(lambda x: get_matchwinner(x), axis=1)
ti_matches_df['team_winner'] = ti_matches_df['winner'].map(ti_teams_df.set_index('team_id').to_dict()['name'])
ti_matches_df['url'] = ti_matches_df['winner'].map(ti_teams_df.set_index('team_id').to_dict()['url'])
ti_matches_df.head()

,match_id,duration,radiant_win,radiant_score,radiant_team_id,dire_score,dire_team_id,total_score,winner,team_winner,url
0,6806663153,2178,False,16,7390454,18,15,34,15,PSG.LGD,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/15.png
1,6806047576,2971,False,23,7732977,24,2586976,47,2586976,OG,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/2586976.png
2,6806638898,3098,False,19,7732977,28,6209804,47,6209804,Royal Never Give Up,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/6209804.png
3,6806648571,2250,True,33,8131728,14,8599101,47,8131728,Hokori,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/8131728.png
4,6806662696,1455,False,5,2586976,34,39,39,39,Evil Geniuses,https://cdn.cloudflare.steamstatic.com/apps/dota2/teamlogos/39.png


In [254]:
radiant_teams = ti_matches_df[['match_id','radiant_team_id','duration']].rename(columns={"radiant_team_id":"team_id"})
dire_teams = ti_matches_df[['match_id','dire_team_id','duration']].rename(columns={"dire_team_id":"team_id"})
ti_teams_matches = pd.concat([radiant_teams,dire_teams],ignore_index=True)
ti_teams_df['avg_length'] = ti_teams_df['team_id'].map(ti_teams_matches.groupby('team_id').mean()['duration'])
# ti_teams_df['avg_length_minutes'] = ti_teams_df.apply(lambda x: convert(x,'avg_length'), axis=1)

In [255]:
hero_stats_df.to_csv('stats/game_player_stats.csv',index=False)
players_df.to_csv('stats/player_stats.csv', index=False)
ti_teams_df.to_csv('stats/team_stats.csv', index=False)
ti_teams_matches.to_csv('stats/game_head_stats.csv', index=False)

In [256]:
hero_list = []
player_list = []
team_list = []
tournament_list = []

# Hero with the most picked:

In [257]:
hero_most_picked = heroes_df[['hero_name','pick_count','url']].dropna().nlargest(3,'pick_count',keep='all')
print(hero_most_picked)
obj = {}
obj["title"] = "Most Picked"
obj["result"] = hero_most_picked.rename(columns={"hero_name":"name","pick_count":"value"}).to_dict('r')
hero_list.append(obj)


        hero_name  pick_count  \
121         Marci        91.0   
10   Shadow Fiend        81.0   
18           Tiny        77.0   

                                                                                          url  
121      https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/marci.png  
10   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/nevermore.png  
18        https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/tiny.png  


# Hero with the most banned:

In [258]:
hero_most_banned = heroes_df[['hero_name','ban_count','url']].dropna().nlargest(3,'ban_count',keep='all')
print(hero_most_banned)
obj = {}
obj["title"] = "Most banned"
obj["result"] = hero_most_banned.rename(columns={"hero_name":"name","ban_count":"value"}).to_dict('r')
hero_list.append(obj)

        hero_name  ban_count  \
31         Enigma      137.0   
122  Primal Beast      124.0   
121         Marci      120.0   

                                                                                             url  
31         https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/enigma.png  
122  https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/primal_beast.png  
121         https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/marci.png  


# Hero with the highest win rate:

In [259]:
hero_max_winrate = heroes_5_df[['hero_name','winrate','url']].dropna().nlargest(3,'winrate',keep='all')
print(hero_max_winrate)
obj = {}
obj["title"] = "Highest Winrate"
obj["result"] = hero_max_winrate.rename(columns={"hero_name":"name","winrate":"value"}).to_dict('r')
hero_list.append(obj)

        hero_name  winrate  \
70     Gyrocopter    85.71   
16   Storm Spirit    78.57   
105  Earth Spirit    71.43   

                                                                                             url  
70     https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/gyrocopter.png  
16   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/storm_spirit.png  
105  https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/earth_spirit.png  


# Hero with the highest average kill:

In [260]:
hero_max_avg_kills = heroes_5_df[['hero_name','avg_kills','url']].dropna().nlargest(3,'avg_kills',keep='all')
print(hero_max_avg_kills)
obj = {}
obj["title"] = "Highest Average Kills"
obj["result"] = hero_max_avg_kills.rename(columns={"hero_name":"name","avg_kills":"value"}).to_dict('r')
hero_list.append(obj)


      hero_name  avg_kills  \
70   Gyrocopter  13.142857   
3   Bloodseeker  12.166667   
33       Sniper  11.409091   

                                                                                           url  
70   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/gyrocopter.png  
3   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bloodseeker.png  
33       https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/sniper.png  


# Hero with the highest average assist:

In [261]:
hero_max_avg_assists = heroes_5_df[['hero_name','avg_assists','url']].dropna().nlargest(3,'avg_assists',keep='all')
print(hero_max_avg_assists)
obj = {}
obj["title"] = "Highest Average Assists"
obj["result"] = hero_max_avg_assists.rename(columns={"hero_name":"name","avg_assists":"value"}).to_dict('r')
hero_list.append(obj)

          hero_name  avg_assists  \
105    Earth Spirit    20.000000   
69   Spirit Breaker    18.444444   
85        Disruptor    17.368421   

                                                                                               url  
105    https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/earth_spirit.png  
69   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/spirit_breaker.png  
85        https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/disruptor.png  


# Hero with the lowest average death:

In [262]:
hero_min_avg_deaths = heroes_5_df[['hero_name','avg_deaths','url']].dropna().nsmallest(3,'avg_deaths',keep='all')
print(hero_min_avg_deaths)
obj = {}
obj["title"] = "Lowest Average deaths"
obj["result"] = hero_min_avg_deaths.rename(columns={"hero_name":"name","avg_deaths":"value"}).to_dict('r')
hero_list.append(obj)

            hero_name  avg_deaths  \
111        Arc Warden    2.285714   
44   Templar Assassin    2.500000   
87         Naga Siren    2.518519   

                                                                                                 url  
111        https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/arc_warden.png  
44   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/templar_assassin.png  
87         https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/naga_siren.png  


# Hero with the highest average last hit:

In [263]:
hero_max_avg_last_hits = heroes_5_df[['hero_name','avg_last_hits','url']].dropna().nlargest(3,'avg_last_hits',keep='all')
print(hero_max_avg_last_hits)
obj = {}
obj["title"] = "Highest Average Last hits"
obj["result"] = hero_max_avg_last_hits.rename(columns={"hero_name":"name","avg_last_hits":"value"}).to_dict('r')
hero_list.append(obj)

       hero_name  avg_last_hits  \
107  Terrorblade     551.142857   
87    Naga Siren     513.407407   
70    Gyrocopter     496.428571   

                                                                                            url  
107  https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/terrorblade.png  
87    https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/naga_siren.png  
70    https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/gyrocopter.png  


# Hero with the highest average xp per minute:

In [264]:
hero_max_avg_xp_per_min = heroes_5_df[['hero_name','avg_xp_per_min','url']].dropna().nlargest(3,'avg_xp_per_min',keep='all')
print(hero_max_avg_xp_per_min)
obj = {}
obj["title"] = "Highest Average XPM"
obj["result"] = hero_max_avg_xp_per_min.rename(columns={"hero_name":"name","avg_xp_per_min":"value"}).to_dict('r')
hero_list.append(obj)

        hero_name  avg_xp_per_min  \
70     Gyrocopter      810.714286   
13          Pudge      755.451613   
39  Faceless Void      751.900000   

                                                                                             url  
70     https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/gyrocopter.png  
13          https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/pudge.png  
39  https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/faceless_void.png  


# Hero with the most kills in a game:

In [265]:
hero_max_max_kills = heroes_df[['hero_name','max_kills','url']].dropna().nlargest(3,'max_kills',keep='all')
print(hero_max_max_kills)
obj = {}
obj["title"] = "Most Kills in a game"
obj["result"] = hero_max_max_kills.rename(columns={"hero_name":"name","max_kills":"value"}).to_dict('r')
hero_list.append(obj)

       hero_name  max_kills  \
3    Bloodseeker       25.0   
10  Shadow Fiend       24.0   
21          Zeus       23.0   

                                                                                           url  
3   https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/bloodseeker.png  
10    https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/nevermore.png  
21         https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/zuus.png  


# Hero with the most last hits in a game:

In [266]:
hero_max_max_last_hits = heroes_df[['hero_name','max_last_hits','url']].dropna().nlargest(3,'max_last_hits',keep='all')
print(hero_max_max_last_hits)
obj = {}
obj["title"] = "Most Last Hits in a game"
obj["result"] = hero_max_max_last_hits.rename(columns={"hero_name":"name","max_last_hits":"value"}).to_dict('r')
hero_list.append(obj)

       hero_name  max_last_hits  \
107  Terrorblade         1372.0   
87    Naga Siren         1196.0   
33        Sniper         1058.0   

                                                                                            url  
107  https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/terrorblade.png  
87    https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/naga_siren.png  
33        https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/heroes/sniper.png  


# Player with the highest kill average:

In [267]:
player_max_avg_kills = players_df[['team_tag','name','avg_kills']].dropna().nlargest(3,'avg_kills',keep='all')
print(player_max_avg_kills)
obj = {}
obj["title"] = "Highest Average Kills"
obj["result"] = player_max_avg_kills.rename(columns={"avg_kills":"value"}).to_dict('r')
player_list.append(obj)

     team_tag       name  avg_kills
2284  THUNDER     Pakazs  10.560000
560    Tundra     skiter   9.727273
2331  THUNDER  DarkMago♥   9.080000


# Player with the highest assist average:

In [268]:
player_max_avg_assists = players_df[['team_tag','name','avg_assists']].dropna().nlargest(3,'avg_assists',keep='all')
print(player_max_avg_assists)
obj = {}
obj["title"] = "Highest Average Assists"
obj["result"] = player_max_avg_assists.rename(columns={"avg_assists":"value"}).to_dict('r')
player_list.append(obj)

     team_tag     name  avg_assists
789    Secret  W_Zayac    17.227273
1414  THUNDER  Matthew    17.120000
656    Tundra    Saksa    16.727273


# Player with the lowest death average:

In [269]:
player_min_avg_deaths = players_df[['team_tag','name','avg_deaths']].dropna().nsmallest(3,'avg_deaths',keep='all')
print(player_min_avg_deaths)
obj = {}
obj["title"] = "Lowest Average Deaths"
obj["result"] = player_min_avg_deaths.rename(columns={"avg_deaths":"value"}).to_dict('r')
player_list.append(obj)

     team_tag    name  avg_deaths
1154   Fnatic   Raven    2.000000
1340    Aster   Monet    2.083333
560    Tundra  skiter    2.272727


# Player with the highest last hit average:

In [270]:
player_max_avg_last_hits = players_df[['team_tag','name','avg_last_hits']].dropna().nlargest(3,'avg_last_hits',keep='all')
print(player_max_avg_last_hits)
obj = {}
obj["title"] = "Highest Average Last Hits"
obj["result"] = player_max_avg_last_hits.rename(columns={"avg_last_hits":"value"}).to_dict('r')
player_list.append(obj)

     team_tag      name  avg_last_hits
2454     BOOM  JACKBOYS     430.173913
1869      RNG     Ghost     421.333333
1748   Hokori   Lumière     419.789474


# Player with the highest GPM average:

In [271]:
player_max_avg_xp_per_min = players_df[['team_tag','name','avg_gold_per_min']].dropna().nlargest(3,'avg_gold_per_min',keep='all')
print(player_max_avg_xp_per_min)
obj = {}
obj["title"] = "Highest Average XPM"
obj["result"] = player_max_avg_xp_per_min.rename(columns={"avg_gold_per_min":"value"}).to_dict('r')
player_list.append(obj)

     team_tag     name  avg_gold_per_min
2284  THUNDER   Pakazs        693.520000
1748   Hokori  Lumière        687.052632
2669  PSG.LGD       萧瑟        676.692308


# Player with the most kills:

In [272]:
player_max_max_kills = players_df[['team_tag','name','max_kills']].dropna().nlargest(3,'max_kills',keep='all')
print(player_max_max_kills)
obj = {}
obj["title"] = "Most Kills in a game"
obj["result"] = player_max_max_kills.rename(columns={"max_kills":"value"}).to_dict('r')
player_list.append(obj)

     team_tag        name  max_kills
1098   Secret  Crystallis       25.0
1748   Hokori     Lumière       24.0
2246     BOOM      Yopaj-       23.0
2284  THUNDER      Pakazs       23.0
2331  THUNDER   DarkMago♥       23.0


# Player with the most assists:

In [273]:
player_max_max_assists = players_df[['team_tag','name','max_assists']].dropna().nlargest(3,'max_assists',keep='all')
print(player_max_max_assists)
obj = {}
obj["title"] = "Most Assists in a game"
obj["result"] = player_max_max_assists.rename(columns={"max_assists":"value"}).to_dict('r')
player_list.append(obj)

     team_tag     name  max_assists
1418     BOOM     TIMS         39.0
1454  BetBoom   RodjER         39.0
30         GG     tOfu         38.0
1783  BetBoom  Noticed         38.0


# Player with the most last hits:

In [274]:
player_max_max_last_hits = players_df[['team_tag','name','max_last_hits']].dropna().nlargest(3,'max_last_hits',keep='all')
print(player_max_max_last_hits)
obj = {}
obj["title"] = "Most Last Hits in a game"
obj["result"] = player_max_max_last_hits.rename(columns={"max_last_hits":"value"}).to_dict('r')
player_list.append(obj)

     team_tag      name  max_last_hits
1869      RNG     Ghost         1372.0
1340    Aster     Monet         1196.0
2454     BOOM  JACKBOYS         1058.0


# Player with the most GPM:

In [275]:
player_max_max_gold_per_min = players_df[['team_tag','name','max_gold_per_min']].dropna().nlargest(3,'max_gold_per_min',keep='all')
print(player_max_max_gold_per_min)
obj = {}
obj["title"] = "Most GPM in a game"
obj["result"] = player_max_max_gold_per_min.rename(columns={"max_gold_per_min":"value"}).to_dict('r')
player_list.append(obj)

     team_tag      name  max_gold_per_min
1539       OG    Yuragi            1034.0
1340    Aster     Monet            1010.0
909        GG  dyrachyo             995.0


# Player with the most different heroes:

In [276]:
player_max_unique_heroes = players_df[['team_tag','name','unique_heroes']].dropna().nlargest(3,'unique_heroes',keep='all')
print(player_max_unique_heroes)
obj = {}
obj["title"] = "Most Different Heroes"
obj["result"] = player_max_unique_heroes.rename(columns={"unique_heroes":"value"}).to_dict('r')
player_list.append(obj)

     team_tag        name  unique_heroes
1539       OG      Yuragi           18.0
940   PSG.LGD  Faith_bian           17.0
2669  PSG.LGD          萧瑟           17.0


# Team with the most kills in a game:

In [277]:
team_max_max_kills = ti_teams_df[['name','max_kills','url']].dropna().nlargest(3,'max_kills',keep='all')
print(team_max_max_kills)
obj = {}
obj["title"] = "Most Kills in a game"
obj["result"] = team_max_max_kills.rename(columns={"max_kills":"value"}).to_dict('r')
team_list.append(obj)

                 name  max_kills
18     Thunder Awaken         66
8        BetBoom Team         57
7   Gaimin Gladiators         54


# Team with the highest kill average:

In [278]:
team_max_avg_kills = ti_teams_df[['name','avg_kills','url']].dropna().nlargest(3,'avg_kills',keep='all')
print(team_max_avg_kills)
obj = {}
obj["title"] = "Highest Average Kills"
obj["result"] = team_max_avg_kills.rename(columns={"avg_kills":"value"}).to_dict('r')
team_list.append(obj)

              name  avg_kills
18  Thunder Awaken  31.560000
3      Team Liquid  29.250000
19     Team Secret  28.772727


# Team with the least deaths in a game:

In [279]:
team_min_min_deaths = ti_teams_df[['name','min_deaths','url']].dropna().nsmallest(3,'min_deaths',keep='all')
print(team_min_min_deaths)
obj = {}
obj["title"] = "Least Deaths in a game"
obj["result"] = team_min_min_deaths.rename(columns={"min_deaths":"value"}).to_dict('r')
team_list.append(obj)

           name  min_deaths
2        Soniqs           3
4       TSM FTX           3
6   Team Spirit           3
15   Team Aster           3
19  Team Secret           3


# Team with the most assists in a game:

In [280]:
team_max_max_assists = ti_teams_df[['name','max_assists','url']].dropna().nlargest(3,'max_assists',keep='all')
print(team_max_max_assists)
obj = {}
obj["title"] = "Most assists in a game"
obj["result"] = team_max_max_assists.rename(columns={"max_assists":"value"}).to_dict('r')
team_list.append(obj)

              name  max_assists
18  Thunder Awaken          160
8     BetBoom Team          157
16          Hokori          140


# Team who wins the longest game:

In [281]:
team_max_duration = ti_matches_df[['team_winner','duration','url']].dropna().nlargest(3,'duration',keep='all')
team_max_duration['duration'] = team_max_duration.apply(lambda x:convert(x['duration']),axis=1)
print(team_max_duration)
obj = {}
obj["title"] = "Wins the longest game"
obj["result"] = team_max_duration.rename(columns={"duration":"value","team_winner":"name"}).to_dict('r')
team_list.append(obj)

             team_winner duration
183               Entity   107:02
102         BetBoom Team    74:33
49   Royal Never Give Up    70:29


# Team who wins the shortest game:

In [282]:
team_min_duration = ti_matches_df[['team_winner','duration','url']].dropna().nsmallest(3,'duration',keep='all')
team_min_duration['duration'] = team_min_duration.apply(lambda x:convert(x['duration']),axis=1)
print(team_min_duration)
obj = {}
obj["title"] = "Wins the shortest game"
obj["result"] = team_min_duration.rename(columns={"duration":"value","team_winner":"name"}).to_dict('r')
team_list.append(obj)


       team_winner duration
120  Evil Geniuses    17:51
181    Team Spirit    19:21
14     Team Secret    19:30


# Team with the highest length average:

In [283]:
team_max_avg_length = ti_teams_df[['name','avg_length','url']].dropna().nlargest(3,'avg_length',keep='all')
team_max_avg_length['avg_length'] = team_max_avg_length.apply(lambda x:convert(x['avg_length']),axis=1)
print(team_max_avg_length)
obj = {}
obj["title"] = "Highest Length Average"
obj["result"] = team_max_avg_length.rename(columns={"avg_length":"value"}).to_dict('r')
team_list.append(obj)

                   name avg_length
17  Royal Never Give Up      45:24
8          BetBoom Team      44:45
14               Fnatic      43:35


# Team with the most different heroes:

In [284]:
team_max_unique_heroes = ti_teams_df[['name','unique_heroes','url']].dropna().nlargest(3,'unique_heroes',keep='all')
print(team_max_unique_heroes)
obj = {}
obj["title"] = "Most Different Heroes"
obj["result"] = team_max_unique_heroes.rename(columns={"unique_heroes":"value"}).to_dict('r')
team_list.append(obj)

          name  unique_heroes
5      PSG.LGD             63
9   beastcoast             57
12          OG             56


# Team with the fewest different heroes:

In [285]:
team_min_unique_heroes = ti_teams_df[['name','unique_heroes','url']].dropna().nsmallest(3,'unique_heroes',keep='all')
print(team_min_unique_heroes)
obj = {}
obj["title"] = "Fewest Different Heroes"
obj["result"] = team_min_unique_heroes.rename(columns={"unique_heroes":"value"}).to_dict('r')
team_list.append(obj)

            name  unique_heroes
8   BetBoom Team             37
11        Entity             37
14        Fnatic             38


# Heroes Picked

In [286]:
ti_heroes_picked = picks_df['hero_id'].nunique()
print(ti_heroes_picked)
obj = {}
obj["title"] = "Heroes Picked"
obj["result"] = ti_heroes_picked
tournament_list.append(obj)

114


# Heroes Banned

In [287]:
ti_heroes_banned = bans_df['hero_id'].nunique()
print(ti_heroes_banned)
obj = {}
obj["title"] = "Heroes banned"
obj["result"] = ti_heroes_banned
tournament_list.append(obj)

105


# Most Total Kills in a game

In [288]:
max_total_kills = ti_matches_df['total_score'].max()
print(max_total_kills)
obj = {}
obj["title"] = "Most Total Kills in a game"
obj["result"] = max_total_kills
tournament_list.append(obj)

99


# Longest Game

In [289]:
max_duration = convert(ti_matches_df['duration'].max())
print(max_duration)
obj = {}
obj["title"] = "Longest Game"
obj["result"] = max_duration
tournament_list.append(obj)

107:02


# Shortest Game

In [290]:
min_duration = convert(ti_matches_df['duration'].min())
print(min_duration)
obj = {}
obj["title"] = "Shortest Game"
obj["result"] = min_duration
tournament_list.append(obj)

17:51


# Most Solo Kills

In [291]:
max_solo_kills = hero_stats_df['kills'].max()
print(max_solo_kills)
obj = {}
obj["title"] = "Most Solo Kills"
obj["result"] = max_solo_kills
tournament_list.append(obj)

25


# Most Solo Deaths

In [292]:
max_solo_deaths = hero_stats_df['deaths'].max()
print(max_solo_deaths)
obj = {}
obj["title"] = "Most Solo Deaths"
obj["result"] = max_solo_deaths
tournament_list.append(obj)

20


# Most Solo Assists

In [293]:
max_solo_assists = hero_stats_df['assists'].max()
print(max_solo_assists)
obj = {}
obj["title"] = "Most Solo Assists"
obj["result"] = max_solo_assists
tournament_list.append(obj)

39


# Highest Solo GPM

In [294]:
max_solo_gold_per_min = hero_stats_df['gold_per_min'].max()
print(max_solo_gold_per_min)
obj = {}
obj["title"] = "Most Solo GPM"
obj["result"] = max_solo_gold_per_min
tournament_list.append(obj)

1034


In [295]:
df1 = pd.DataFrame(hero_list)
df2 = pd.DataFrame(player_list)
df3 = pd.DataFrame(team_list)
df4 = pd.DataFrame(tournament_list)
df1.to_json('hero_list.json',orient='records')
df2.to_json('player_list.json',orient='records')
df3.to_json('team_list.json',orient='records')
df4.to_json('tournament_list.json',orient='records')

In [296]:
import convert
convert.create_readme()